<a href="https://colab.research.google.com/github/toskn/neo4j_db/blob/main/%D0%92%D0%BE%D1%80%D0%B4%D0%BD%D0%B5%D1%82_%D0%BD%D0%B0_%D0%BA%D0%BE%D0%BB%D0%B5%D0%BD%D0%BA%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Гиперонимия

Скачивам таблицу <гипоним гипероним частота-излечения-отношения> [отсюда](https://nlpub.ru/Russian_Distributional_Thesaurus)

In [ ]:
!wget http://panchenko.me/data/joint/isas/ru-librusec-wiki-diff.csv.gz

--2021-03-29 13:52:42--  http://panchenko.me/data/joint/isas/ru-librusec-wiki-diff.csv.gz
Resolving panchenko.me (panchenko.me)... 130.104.253.4
Connecting to panchenko.me (panchenko.me)|130.104.253.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15706045 (15M) [application/x-gzip]
Saving to: ‘ru-librusec-wiki-diff.csv.gz’

ru-librusec-wiki-di 100%[===================>]  14.98M  4.35MB/s    in 3.5s    

2021-03-29 13:52:45 (4.31 MB/s) - ‘ru-librusec-wiki-diff.csv.gz’ saved [15706045/15706045]



Читаем ее в датафрейм

In [ ]:
import pandas as pd

In [ ]:
raw_data = pd.read_csv('ru-librusec-wiki-diff.csv.gz', compression='gzip',
                   error_bad_lines=False, sep='\t')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1597651 entries, 0 to 1597650
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   hyponym   1597650 non-null  object
 1   hypernym  1597651 non-null  object
 2   freq      1597651 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 36.6+ MB


Возьмем только пары с частотой больше 10, чтобы уменьшить объем данных и выкинуть какие-то очевидно редкие случаи

In [ ]:
data = raw_data[raw_data.freq >= 10]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72732 entries, 10 to 1597626
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hyponym   72732 non-null  object
 1   hypernym  72732 non-null  object
 2   freq      72732 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [ ]:
data.head()

,hyponym,hypernym,freq
10,грызун,животное,60
11,грызун,животных,24
12,грызун,млекопитающее,22
13,грызун,кошка,18
14,грызун,кошками,18


Посмотрим на количество уникальных значений

In [ ]:
pon = set(data.hyponym)
len(pon)

19976

In [ ]:
per = set(data.hypernym)
len(per)

9243

Отфильтруем слова, которые есть ив гипонимах, и в гиперонимах

In [ ]:
both = [x for x in per if x in pon]
len(both)

2540

In [ ]:
pip install pymorphy2

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 8.2MB 13.4MB/s 


In [ ]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [ ]:
from collections import Counter

In [ ]:
Counter([morph.parse(b)[0].tag.POS for b in both]).most_common()

[('NOUN', 2457),
 ('ADJF', 40),
 ('INFN', 12),
 ('ADVB', 7),
 ('VERB', 6),
 ('PREP', 6),
 ('PRTF', 4),
 ('ADJS', 3),
 ('PRCL', 2),
 ('GRND', 1),
 ('COMP', 1),
 ('CONJ', 1)]

Напишем функцию, чтобы отфильровать среди свществительных и прилагательных только слова в ед.ч. и номинативе, и убрать все словосочетания

In [ ]:
def filter_nom(word):
    if len(word.split(' ')) > 1:
        return False
    p = morph.parse(word)[0]

    if p.tag.POS not in ["NOUN", "ADJF"]:
        return True

    if p.tag.case is None:
        return False

    if p.tag.case != 'nomn':
        return False
    
    if p.tag.number != 'sing':
        return False
    
    return True

In [ ]:
filter_nom('кошка')

True

In [ ]:
filter_nom('кошкой')

False

In [ ]:
filter_nom('кошки')

False

In [ ]:
filter_nom('плавать')

True

In [ ]:
filter_nom('быстрый')

True

Отфильтруем слова еще раз с помощью нашей функции

In [ ]:
noms = [x for x in both if filter_nom(x)]
len(noms)

1307

In [ ]:
print(noms[:100])

['идеал', 'мышца', 'желудок', 'богатство', 'плащ', 'гигант', 'причина', 'случайность', 'обычай', 'уважение', 'полет', 'сад', 'восхищение', 'таран', 'специализация', 'отношение', 'палатка', 'дочь', 'микроб', 'мед', 'колесница', 'нападение', 'устройство', 'дешевка', 'быт', 'каша', 'раса', 'национальность', 'баня', 'проем', 'отвращение', 'слой', 'свидетель', 'свобода', 'событие', 'передача', 'тоска', 'насекомое', 'ткань', 'озеро', 'капитал', 'корабль', 'потеря', 'кислота', 'подметка', 'лучник', 'бич', 'поведение', 'славянин', 'экипаж', 'служба', 'пушка', 'корень', 'змея', 'режим', 'перстень', 'негр', 'пан', 'оформление', 'катаклизм', 'автобиография', 'слава', 'папоротник', 'идеализм', 'выбор', 'сухарь', 'глагол', 'сон', 'быть', 'творчество', 'обмундирование', 'военнопленный', 'старость', 'ужин', 'провокация', 'спутник', 'жених', 'недомогание', 'партия', 'антибиотик', 'древесина', 'покупка', 'затрата', 'пища', 'мать', 'обращение', 'бедняк', 'сюрприз', 'лозунг', 'ботаник', 'трофей', 'сочета

Данных получается достаточно много, возьмем только те пары, которые встретились 30 и более раз. Заодно выбросим все дубликаты, если они вдруг есть

In [ ]:
sgnom_data = data[data.hyponym.isin(noms) & data.hypernym.isin(noms)]
res = sgnom_data[sgnom_data.freq >= 30]
res.drop_duplicates(inplace=False)
res

,hyponym,hypernym,freq
10,грызун,животное,60
190,результат,образ,38
635,мясо,продукт,1196
638,мясо,пища,168
640,мясо,сочетание,98
...,...,...,...
1590333,учитель,специалист,48
1590334,учитель,работник,38
1594746,теория,наука,72
1594747,теория,дисциплина,30


Посчитаем количество узлов

In [ ]:
nodes = set(res.hyponym) | set(res.hypernym)
len(nodes)

693

In [ ]:
nodes_dict = {n: {'lemma': n,
                  'pos': morph.parse(n)[0].tag.POS} 
              for n in nodes}
[x for x in nodes_dict.values() if x['pos'] != "NOUN"]

[{'lemma': 'ночью', 'pos': 'ADVB'},
 {'lemma': 'домашний', 'pos': 'ADJF'},
 {'lemma': 'напасть', 'pos': 'INFN'},
 {'lemma': 'красный', 'pos': 'ADJF'},
 {'lemma': 'сегодня', 'pos': 'ADVB'},
 {'lemma': 'уже', 'pos': 'ADVB'},
 {'lemma': 'ж', 'pos': 'PRCL'},
 {'lemma': 'к', 'pos': 'PREP'},
 {'lemma': 'служить', 'pos': 'INFN'},
 {'lemma': 'уж', 'pos': 'ADVB'},
 {'lemma': 'добро', 'pos': 'ADVB'},
 {'lemma': 'холодный', 'pos': 'ADJF'},
 {'lemma': 'ли', 'pos': 'CONJ'},
 {'lemma': 'с', 'pos': 'PREP'},
 {'lemma': 'страсть', 'pos': 'ADVB'},
 {'lemma': 'горючее', 'pos': 'ADJF'},
 {'lemma': 'до', 'pos': 'PREP'},
 {'lemma': 'добрый', 'pos': 'ADJF'},
 {'lemma': 'у', 'pos': 'PREP'},
 {'lemma': 'стать', 'pos': 'INFN'},
 {'lemma': 'были', 'pos': 'VERB'},
 {'lemma': 'черный', 'pos': 'ADJF'},
 {'lemma': 'пресмыкаться', 'pos': 'INFN'},
 {'lemma': 'быть', 'pos': 'INFN'},
 {'lemma': 'дать', 'pos': 'INFN'},
 {'lemma': 'легкий', 'pos': 'ADJF'}]

In [ ]:
word_nodes = pd.DataFrame([x for x in nodes_dict.values()])
word_nodes.head()

,lemma,pos
0,личность,NOUN
1,карьера,NOUN
2,фермент,NOUN
3,книжка,NOUN
4,белье,NOUN


Сохраним в файлы

In [ ]:
res.to_csv('hyponyms.csv', index=False, sep='\t', header=['Source', 'Target', 'Freq'])

In [ ]:
word_nodes.to_csv('word_nodes.csv', index=False, sep='\t', header=['Lemma', 'POS'])

In [ ]:
import json

In [ ]:
with open('word_nodes.json', 'w', encoding='utf-8') as f:
    json.dump(nodes_dict, f, ensure_ascii=False, indent=2)

# Смыслы

In [ ]:
!wget http://panchenko.me/data/joint/ru/ddt-ru150g_trigram__FreqSigLMI__PruneContext_s_0.0_w_10_f_10_wf_0_wpfmax_1000_wpfmin_2_p_1000__AggrPerFt__SimCount_sc_one_ac_False__SimSortlimit_200_minsim_2-cw-e0-N200-n200-minsize5.csv.gz

--2021-03-29 16:34:48--  http://panchenko.me/data/joint/ru/ddt-ru150g_trigram__FreqSigLMI__PruneContext_s_0.0_w_10_f_10_wf_0_wpfmax_1000_wpfmin_2_p_1000__AggrPerFt__SimCount_sc_one_ac_False__SimSortlimit_200_minsim_2-cw-e0-N200-n200-minsize5.csv.gz
Resolving panchenko.me (panchenko.me)... 130.104.253.4
Connecting to panchenko.me (panchenko.me)|130.104.253.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 917204524 (875M) [application/x-gzip]
Saving to: ‘ddt-ru150g_trigram__FreqSigLMI__PruneContext_s_0.0_w_10_f_10_wf_0_wpfmax_1000_wpfmin_2_p_1000__AggrPerFt__SimCount_sc_one_ac_False__SimSortlimit_200_minsim_2-cw-e0-N200-n200-minsize5.csv.gz’

ddt-ru150g_trigram_ 100%[===================>] 874.71M  4.76MB/s    in 3m 1s   

2021-03-29 16:37:49 (4.84 MB/s) - ‘ddt-ru150g_trigram__FreqSigLMI__PruneContext_s_0.0_w_10_f_10_wf_0_wpfmax_1000_wpfmin_2_p_1000__AggrPerFt__SimCount_sc_one_ac_False__SimSortlimit_200_minsim_2-cw-e0-N200-n200-minsize5.csv.gz’ saved [917204524/9

In [ ]:
filename = 'ddt-ru150g_trigram__FreqSigLMI__PruneContext_s_0.0_w_10_f_10_wf_0_wpfmax_1000_wpfmin_2_p_1000__AggrPerFt__SimCount_sc_one_ac_False__SimSortlimit_200_minsim_2-cw-e0-N200-n200-minsize5.csv.gz'

raw_meanings = pd.read_csv(filename, compression='gzip',
                   error_bad_lines=False, sep='\t')

In [ ]:
raw_meanings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2646087 entries, 0 to 2646086
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   word     object 
 1   cid      int64  
 2   cluster  object 
 3   isas     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 80.8+ MB


In [ ]:
meanings = raw_meanings[raw_meanings.word.isin(nodes)]
meanings.sort_values(by=['word', 'cid'])

,word,cid,cluster,isas
148318,автомобиль,0,"джип:309.0,грузовик:297.0,фургон:242.0,автобус...",NaN
431760,автор,0,"издатель:33.0,писатель:24.0,переводчик:23.0,по...",NaN
431761,автор,3,"сборник:30.0,сборники:24.0,серия:21.0,цикл:17....",NaN
431762,автор,4,"создатель:80.0,создателя:23.0,основатель:22.0,...",NaN
431763,автор,5,"составитель:101.0,автора:86.0,автором:83.0,авт...",NaN
...,...,...,...,...
797460,язва,1,"астма:46.0,болезнь:45.0,подагра:40.0,аллергия:...",NaN
797461,язва,2,"рана:109.0,ранка:49.0,ссадина:44.0,опухоль:40....",NaN
797462,язва,3,"бойня:27.0,трагедия:27.0,вражда:23.0,драма:22....",NaN
797463,язва,4,"несварение:64.0,расстройство:44.0,промывание:3...",NaN


In [ ]:
def get_len(cluster):
    return len(cluster.split(','))

In [ ]:
meanings['count'] = meanings.cluster.apply(get_len)
meanings['count'].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


count    1289.000000
mean      102.060512
std        77.399420
min         5.000000
25%        21.000000
50%        95.000000
75%       195.000000
max       199.000000
Name: count, dtype: float64

In [ ]:
meanings_edges = pd.DataFrame(columns=['Word', 'MeaningId'])
meaning_nodes_dict = dict()

for word, group in meanings.groupby(by='word'):
    n = 0
    for _, row in group.iterrows():
        n += 1
        meaning_id = f'{word}.{n}'
        top5 = [t.split(':')[0] for t in row['cluster'].split(',')[:5]]

        meanings_edges = meanings_edges.append({'Word': word,
                                                'MeaningId': meaning_id}, 
                                               ignore_index=True)
        
        meaning_nodes_dict[meaning_id] = {'MeaningId': meaning_id, 'Top5': ', '.join(top5)}

meanings_edges.head()

,Word,MeaningId
0,автомобиль,автомобиль.1
1,автор,автор.1
2,автор,автор.2
3,автор,автор.3
4,автор,автор.4


In [ ]:
len(meaning_nodes_dict)

1289

In [ ]:
meaning_nodes = pd.DataFrame([x for x in meaning_nodes_dict.values()])
meaning_nodes.head()

,MeaningId,Top5
0,автомобиль.1,"джип, грузовик, фургон, автобус, лимузин"
1,автор.1,"издатель, писатель, переводчик, поэт, художник"
2,автор.2,"сборник, сборники, серия, цикл, подборка"
3,автор.3,"создатель, создателя, основатель, владелец, ин..."
4,автор.4,"составитель, автора, автором, автору, авторы"


In [ ]:
meanings_edges.to_csv('meanings.csv', index=False, sep='\t', header=['Word', 'MeaningId'])

In [ ]:
meaning_nodes.to_csv('meaning_nodes.csv', index=False, sep='\t', header=['MeaningId', 'Top5'])

In [ ]:
# with open('meanings_nodes.json', 'w', encoding='utf-8') as f:
#     json.dump(meaning_nodes, f, ensure_ascii=False, indent=2)

# Bulk создание базы данных

In [ ]:
# Создание всего из csv файлов

"""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/toskn/neo4j_db/main/Data/word_nodes.csv' AS line FIELDTERMINATOR '\t'
CREATE (:Word {lemma: line.Lemma, pos: line.POS})
"""

"""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/toskn/neo4j_db/main/Data/meaning_nodes.csv' AS line FIELDTERMINATOR '\t'
CREATE (:Meaning {id: line.MeaningId, Top5: line.Top5})
"""

"""
LOAD CSV WITH HEADERS FROM 'https://github.com/toskn/neo4j_db/raw/main/Data/hyponyms.csv' AS row FIELDTERMINATOR '\t'
MATCH (s:Word {lemma: row.Source})
MATCH (t:Word {lemma: row.Target})
MERGE (s)-[:IS_HYPONYM_OF {freq: toInteger(row.Freq)}]->(t)
"""

"""
LOAD CSV WITH HEADERS FROM 'https://github.com/toskn/neo4j_db/raw/main/Data/meanings.csv' AS row FIELDTERMINATOR '\t'
MATCH (s:Word {lemma: row.Word})
MATCH (t:Meaning {id: row.MeaningId})
CREATE (s)-[:HAS_MEANING]->(t)
"""

# Запросы

In [ ]:
# запросы

## все гипонимы слова напиток
"""
MATCH (x:Word {lemma: 'напиток'})<-[:IS_HYPONYM_OF]-(word)
RETURN word.lemma
"""

## посчитать кол-во гипонимов слова товар
"""
MATCH (n:Word) <-[r]- (word) WHERE n.lemma='товар'
RETURN count(word)
"""
## топ слов с максимальным кол-вом гиперонимов
"""
MATCH (n: Word)-[:IS_HYPONYM_OF]->(word)
WITH n, count(word) AS hyperonims
ORDER BY hyperonims DESC
LIMIT 10
RETURN n.lemma, n.pos, hyperonims
"""

## самое многозначное слово
"""
MATCH (n: Word)-[:HAS_MEANING]-(meaning)
WITH n, count(meaning) AS polysem
ORDER BY polysem DESC
RETURN n.lemma
LIMIT 1
"""
## и все его значения
"""
MATCH (n: Word)-[:HAS_MEANING]->(meaning)
WITH n, count(meaning) AS polysem
ORDER BY polysem DESC
LIMIT 1
WITH n.lemma as target 
MATCH (n: Word {lemma: target})-[:HAS_MEANING]->(meaning)
RETURN n, meaning
"""

## То же самое, только не узлы, а значения
"""
MATCH (n: Word)-[:HAS_MEANING]->(meaning)
WITH n, count(meaning) AS polysem
ORDER BY polysem DESC
LIMIT 1
WITH n.lemma as target 
MATCH (n: Word {lemma: target})-[:HAS_MEANING]->(meaning)
RETURN n.lemma, meaning.Top5
"""

## слова без значения
"""
MATCH (n:Word)
WHERE NOT (n)-[:HAS_MEANING]-()
RETURN n.lemma
"""

## гипонимы слова товар, которые встретились с ним более 100 раз
"""
MATCH (n:Word {lemma: 'товар'})<-[r]-(word)
WHERE r.freq > 100
RETURN n, word
"""

## Список частей речи
"""
MATCH (n:Word)
RETURN DISTINCT n.pos
"""

## посчитать кол-во узлов каждой из частей речи и вернуть в порядке убывания
"""
MATCH (n:Word)
RETURN n.pos, count(*) AS count_pos
ORDER BY count_pos DESC
"""

## слова содержашие -ться- и их соседи (и слова, и значения)
"""
MATCH (n:Word)--(word)
WHERE n.lemma CONTAINS 'ться'
RETURN n, word
"""

## все многозначные эмоции и их значения
"""
MATCH (:Word {lemma: 'эмоция'})<-[:IS_HYPONYM_OF]-(emotion)-[:HAS_MEANING]->(meaning)
WITH emotion, count(meaning) AS meanings
WHERE meanings > 1
RETURN (emotion) --> (:Meaning)
"""

## все многозначные растения и их значения
"""
MATCH (:Word {lemma: 'растение'})<-[:IS_HYPONYM_OF]-(word {pos: 'NOUN'})-[:HAS_MEANING]->(meaning)
WITH word, count(meaning) AS meanings
WHERE meanings > 1
RETURN (word) --> (:Meaning)
"""

In [ ]:
# запросы на графовую структуру ай гесс

## ближайший путь от XXX к пулемету (такого нет)
"""
OPTIONAL MATCH
  (chai:Word {lemma: 'XXX'}),
  (gun:Word {lemma: 'пулемет'}),
  p = shortestPath((chai)-[*..15]-(gun))
RETURN p
"""

## ближайший путь от чая к пулемету
"""
MATCH
  (chai:Word {lemma: 'чай'}),
  (gun:Word {lemma: 'пулемет'}),
  p = shortestPath((chai)-[*..30]-(gun))
RETURN p
"""

## пути от автомобиля до ярости длиной от 1 до 5 где все узлы - существительные
"""
MATCH p = (a)-[*1..5]-(b)
WHERE
  a.lemma = 'автомобиль'
  AND b.lemma = 'ярость'
  AND all(x IN nodes(p) WHERE x.pos = 'NOUN')
RETURN p
"""

## все пути длиной до 4, содержащие чай 1 раз
"""
MATCH p = ()-[*1..4]->()
WHERE
 single(var IN nodes(p) WHERE var.lemma = 'чай')
RETURN p
"""

## среднее значение полисемии (по факту - среднее значение out degree)
"""
MATCH (n: Word)-[:HAS_MEANING]-(meaning)
WITH n, count(meaning) AS polysem
RETURN avg(polysem)
"""

## разделить на сообщества
"""
CALL gds.graph.create(
    'Communities',
    'Word',
    'IS_HYPONYM_OF',
    {
        relationshipProperties: 'freq'
    }
)
"""

"""
CALL gds.labelPropagation.stream('Communities')
YIELD nodeId, communityId AS Community
SET gds.util.asNode(nodeId).community = Community
RETURN  gds.util.asNode(nodeId)
"""

## посчитать коээфициент кластеризации
"""
CALL gds.graph.create(
  'myGraph',
  'Word',
  {
    IS_HYPONYM_OF: {
      orientation: 'UNDIRECTED'
    }
  }
)
"""

"""
CALL gds.localClusteringCoefficient.mutate('myGraph', {
  mutateProperty: 'localClusteringCoefficient'
})
YIELD averageClusteringCoefficient, nodeCount
"""

"""
CALL gds.localClusteringCoefficient.stream('myGraph')
YIELD nodeId, localClusteringCoefficient
RETURN gds.util.asNode(nodeId).lemma AS name, localClusteringCoefficient
ORDER BY localClusteringCoefficient DESC
"""

## прочее
"""
CALL db.schema.visualization()
"""

Анна Полянская, 2021